# 3D Perception Project Writeup

## For the Udacity Robotics Engineer Nanodegree

## By Harry Turner on 5th March

---------------------------------------------------------------------------------------------------

# Writeup

This write up explains the perception pipeline in detail. It also displays the results of testing the pipeline on three different test worlds. 

It address each of the points in the rubric, and offers supporting images and statements in each section. 

# Perception Pipeline

## Raw Point Cloud

The input from the RGBD camera is a point cloud, where every point has XYZ coordinates, and RGB colour information. Before I do any processing, I will visualise the point cloud without any noise, to see what I'm working with. This is, effectively, what the robot is "seeing".

I used rViz to visualise the point cloud, the output is shown in the image below.

You can see a selection of objects on the table, the resolution is so small that you can't even see the points. But this give me a pretty good idea of the sort of objects that my robot will need to recognise.

![raw.png](images/raw.png)

## Noisy Input

The actual input from the RGBD camera is noisy, this is important to recognise as it can be very detrimental to further processing techniques if not handled properly! 

I visualised the noisy point cloud in rViz, note that the points are represented as spheres now, to help see the noisy points.

You can see a significant amount of noise surrounding the objects on the table! 

![noise.png](images/noise.png)

## Noise Filter

The noise filter is important as it removes the noise from the input by treating them as statistical outliers. The noise filter itself is a statistical outlier filter that "finds" outliers and removes them. To find any outlier, it goes through every point and looks to see whether it's a certain distance away from it's neighbours.

I implemented the noise filter using the PCL library, demonstrated in the code below. I set the values of k, and x by experiment. I found that these values reduced the amount of noise without losing any detail on the objects.

The output from the noise filter is a point cloud with the noise removed. This can be seen in the image below. You can see that noise has been reduced significantly, this will make further processing steps easier.

In [ ]:
    # Statistical Outlier Filtering
    ###############################

    # Create a outlier filter object for our input point cloud.
    outlier_filter = cloud.make_statistical_outlier_filter()

    # Set the number of neighboring points to analyze for any given point
    outlier_filter.set_mean_k(10)

    # Set threshold scale factor
    x = 0.01

    # Any point with a mean distance larger than global (mean distance+x*std_dev) will be considered outlier
    outlier_filter.set_std_dev_mul_thresh(x)

    # Finally call the filter function for magic
    cloud_filtered = outlier_filter.filter() 

![no_noise.png](images/no_noise.png)

## Voxel Downsizing

Voxel Downsizing is a method of reducing the number of points in the point cloud, by reducing the resolution of the points. The importance of this is that it significantly speeds up future processing steps! 

To achieve this, I implemented a voxel downsizing filter using the PCL library. The code for this implementation is shown below. I chose a leaf size of 0.01. This was found by experimentation to be the value that reduced the number of points significantly, whilst not loosing any important features.

The result is shown in an image at the end of the "RANSAC Filter" section. You can see from the image that the resolution of the objects is much reduced, although they are still clearly identifiable which is important.

In [ ]:
    # Voxel Grid filter
    ###################

    # Create a VoxelGrid filter object for our input point cloud.
    vox = cloud_filtered.make_voxel_grid_filter()

    # Leaf size.
    LEAF_SIZE = 0.01

    # Set the leaf size.
    vox.set_leaf_size(LEAF_SIZE, LEAF_SIZE, LEAF_SIZE)

    # Call the filter function to obtain the resultant downsampled point.
    cloud_filtered = vox.filter()


## Pass Through Filter

The pass through filter is a method of specifying a region of interest. All of the objects are on top of a table, so we don't care about anything under the table. Therefore, we can set a region of interest to be above the table, filtering out everything else. This is important for reducing the search space for later techniques. I found that I needed another filter to remove the boxes, which can actually be seen in the images above. Otherwise, they would confuse the object recognition algorithm.

I implemented two filters, one in the Z axis, to filter out points below the table, and one in the X axis, to filter out the edges of the boxes. The values for the min and max were found by inspecting the scene and determining what region the objects of interest were in.

The results are shown in the image at the end of the "RANSAC Filter" section. You can see from this image that the box edges have been removed.

In [ ]:
    # PassThrough filter Z
    ######################

    # Create a PassThrough filter object.
    passthrough = cloud_filtered.make_passthrough_filter()

    # Assign axis and range to the passthrough filter object.
    filter_axis = 'z'
    passthrough.set_filter_field_name(filter_axis)
    axis_min = 0.6
    axis_max = 0.8
    passthrough.set_filter_limits(axis_min, axis_max)

    # Use the filter function to obtain the resultant point cloud.
    cloud_filtered = passthrough.filter()


    # PassThrough filter X
    ######################

    # Create a PassThrough filter object.
    passthrough = cloud_filtered.make_passthrough_filter()

    # Assign axis and range to the passthrough filter object.
    filter_axis = 'x'
    passthrough.set_filter_field_name(filter_axis)
    axis_min = 0.35
    axis_max = 0.8
    passthrough.set_filter_limits(axis_min, axis_max)

    # Use the filter function to obtain the resultant point cloud.
    cloud_filtered = passthrough.filter()

## RANSAC Filter

The RANSAC filter is a method to find objects that meet a certain model. In this instance, I am using it to find objects that belong to a plane, because they are most likely to belong to the table! Therefore, I can find the table in the point cloud, by using RANSAC plane segmentation. This is important, because I can then remove the plane, and keep the objects behind, and the resulting point cloud will contain just the objects.

I implemented the RANSAC plane segmentation with the PCL library, the code can be seen below. I found the value of max_distance by inspection and experimentation.

The results can be seen in the image that follows the code. You can see three things here. First, the resolution is much reduced from the voxel downsizing. Secondly, the boxes have been removed from the pass through filter. Thirdly, the table has been removed by the RANSAC Filter. The objects are the only things left in the point cloud, which is now ready for clustering segmentation.

In [ ]:
    # RANSAC plane segmentation
    ###########################

    # Create the segmentation object.
    seg = cloud_filtered.make_segmenter()

    # Set the model you wish to fit.
    seg.set_model_type(pcl.SACMODEL_PLANE)
    seg.set_method_type(pcl.SAC_RANSAC)

    # Max distance for a point to be considered fitting the model.
    MAX_DISTANCE = 0.01
    seg.set_distance_threshold(MAX_DISTANCE)

    # Call the segment function.
    inliers, coefficients = seg.segment()
    
    # Extract objects
    #################

    # Extract outliers
    cloud_objects = cloud_filtered.extract(inliers, negative=True)

![objects.png](images/objects.png)

## Clustering Segmentation

Clustering is a method of grouping similar sets of points. In this case, I use the DBSCAN clustering algorithm, to find clusters of points within the point cloud, based on their XYZ values. The idea is that the clusters each represent an object, so by clustering the point cloud, I can segment it to find the objects. This is important so that I can use an object recognition algorithm on each of the clusters/objects in the next step.

I implemented clustering using the PCL library, the code for this can be seen below. First I had to convert the points to XYZ, and then convert the points to a K-D tree. This is to make the algorithm more efficient by searching for neighbouring points. I found the values of cluster tolerance, minimum cluster size, and maximum cluster size by experiment. Since my objects were the only things in the scene, the largest cluster size needed to be larger than the number of points in the largest object. 

The results can be seen in the image below the code. In this case, each cluster has been coloured, so you can see that the clustering algorithm has identified the clusters to be the individual objects. Therefore the clustering algorithm has been successful, I can now extract the points for each cluster and pass these to the object recognition algorithm.

In [ ]:
    # Euclidean Clustering
    ######################

    # Turn to XYZ
    white_cloud = XYZRGB_to_XYZ(cloud_objects)

    # Construct K-D Tree
    tree = white_cloud.make_kdtree()

    # Create cluster extraction object.
    ec = white_cloud.make_EuclideanClusterExtraction()

    # Set cluster parameters
    ec.set_ClusterTolerance(0.05)
    ec.set_MinClusterSize(10)
    ec.set_MaxClusterSize(5000)

    # Search the K-D Tree for clusters
    ec.set_SearchMethod(tree)

    # Extract indices for each of the discovered clusters
    cluster_indices = ec.Extract()

![clusters.png](images/clusters.png)

## Object Detection

Object detection is where I attempt to classify the clusters as a particular class of object. To do this, I'll use a machine learning algorithm called a Support Vector Machine. This requires training on a training set of objects. Once it's trained, it can be deployed in the perception pipeline. The features I'll input to the SVM are the normal signature for the cluster, and the colour signature. The signature in this case will be the histogram, constructed by finding the total distribution of colours and normals, and converting to them to histograms.

### Training

To train my SVM, I needed training data. 

I ran the "capture_features.py" script, with the number of instances per object set to 30. I also specified a HSV colour space for the histogram. Having 30 examples of each object, meant that my model would see the same object 30 different times, in different orientations, therefore it would hopefully learn robust features for that object. I used the HSV colour space as it's less sensitive to light. I then trained my model with the "train_svm.py" script, and outputted the results of it's accuracy, in the form of a confusion matrix. This can be seen below.

The confusion matrix shows that the classifier is doing a good job of classifying the test data, as most of the weight falls along the diagonal. It gets the occasional classification wrong, for example, it has misclassified the glue for the soap. Overall, this classifier is strong enough to implement in my object detection pipeline. 

![figure_1.png](images/figure_2.png)

### Object Detection Code

Object detection is the step where my SVM model attempts to classify the extracted objects from the clustering algorithm in the step before. This is important, because if it can classify the clusters as belonging to a particular class, then the robot has identified an object!

To achieve this, I use my model to predict a class, using SKLEARN. I do this for every cluster in the list, and add each of the "found" objects to a new list. This new list is then output as the result.

To see the results of the object detection, there is a line of code that outputs a label for each cluster, and publishes it to a topic. This label can then be visualised in rViz. In the image that follows the code, you can see the labels added to each of the clusters for the objects in the scene. It can be tricky to determine which label goes with which object, but you can see that the detection algorithm correctly classifies every object with the exception of the sticky notes, which it thinks are snacks. This is a successful implementation of the detection algorithm, and is ready to be deployed in the tests.

In [ ]:
    # Detect Objects
    ################

    detected_objects_labels = []
    detected_objects = []

    for index, pts_list in enumerate(cluster_indices):

        # Grab the points for the cluster from the extracted outliers (cloud_objects)
        pcl_cluster = cloud_objects.extract(pts_list)

        # Convert the cluster from pcl to ROS using helper function
        ros_cluster = pcl_to_ros(pcl_cluster)

        # Extract histogram features
        chists = compute_color_histograms(ros_cluster, using_hsv=True)
        normals = get_normals(ros_cluster)
        nhists = compute_normal_histograms(normals)
        feature = np.concatenate((chists, nhists))

        # Make the prediction, retrieve the label for the result
        # and add it to detected_objects_labels list
        prediction = clf.predict(scaler.transform(feature.reshape(1,-1)))
        label = encoder.inverse_transform(prediction)[0]
        detected_objects_labels.append(label)

        # Publish a label into RViz
        label_pos = list(white_cloud[pts_list[0]])
        label_pos[2] += .4
        object_markers_pub.publish(make_label(label,label_pos, index))

        # Add the detected object to the list of detected objects.
        do = DetectedObject()
        do.label = label
        do.cloud = ros_cluster
        detected_objects.append(do)

![labels.png](images/labels.png)

# Testing

## Test 1

Test 1 was successful, the pipeline classified all three objects correctly. See the image below for reference.

![world-1.png](images/world-1.png)

## Test 2

Test 2 was partially successful, the pipeline classified four out of five objects correctly. It misclasifies the book as snacks. See the image below for reference.

![world-2.png](images/world-2.png)

## Test 3

Test 3 was successful, the pipeline classified eight out of eight objects correctly. See the image below for reference.

![world-3.png](images/world-3.png)

# Improvements

To improve the pipeline, I could have used other machine learning algorithms instead of an SVM. For example, I could have used a deep neural network, which may have given better performance. 

I could also have used more features for the objects, for example, size of object. More features, may have helped my robot identify objects more easily.

# Limitations

One important limitation in this pipeline, is that it requires objects to be seperated. If the glue was lieing on top of the snacks for example, then my clustering algorithm would cluster them together. The resulting features would be different, and the object recognition algorithm wouldn't be able to classify the object, it certainly wouldn't realise there were two objects there!